In [1]:
import asyncio
import pandas
import yahoo_fin.stock_info as si

In [2]:
class Company:
    def __init__(self, symbol):
        self.symbol = symbol
        self.fundamental_indicators = {}

In [3]:
def to_float(val):
    if val == 0:
        return float(0)

    val = str(val).upper()

    if '%' in val:
        return round(float(val[:-1]), 4)

    m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}

    for key in m.keys():
        if key in val:
            multiplier = m.get(val[-1])
            return round(float(val[:-1]) * (10 ** multiplier), 4)
    return round(float(val), 4)

In [4]:
def get_statatistics(symbol):
    url = f"https://finance.yahoo.com/quote/{symbol}/key-statistics?p={symbol}"
    dataframes = pandas.read_html(url)
    return pandas.concat(dataframes[1:])


In [5]:
def get_data_item(result, dataframe, columns):
    for column_to_find, column_to_name in columns.items():
        try:
            result[column_to_name] = list((dataframe.loc[dataframe[0] == column_to_find].to_dict()[1]).values())[0]
        except Exception as ex:
            result[column_to_name] = 'NA'


In [6]:
def get_last_data_item(result, dataframe, columns):
    data = dataframe.iloc[:, :2]
    data.columns = ["Column", "Last"]

    for column_to_find, column_to_name in columns.items():
        try:
            val = data[data.Column.str.contains(column_to_find, case=False, regex=True)].iloc[0, 1]
            float_val = to_float(val)
            result[column_to_name] = float_val
        except Exception as ex:
            result[column_to_name] = "NA"

In [15]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [16]:
async def get_fundamental_indicators_for_company(config, company):
    company.fundmantal_indicators = {}

    # Statistics Valuation
    keys = {
        'Market Cap (intraday) 5': 'MarketCap',
        'Price/Sales (ttm)': 'PS',
        'Trailing P/E': 'PE',
        'PEG Ratio (5 yr expected) 1': 'PEG',
        'Price/Book (mrq)': 'PB'
     }
    data = si.get_stats_valuation(company.symbol)
    get_data_item(company.fundmantal_indicators, data, keys)

    # Income statement and Balance sheet
    data = get_statatistics(company.symbol)

    get_data_item(company.fundmantal_indicators, data,
                  {
                      'Profit Margin': 'ProfitMargin',
                      'Operating Margin (ttm)': 'OperMargin',
                      'Current Ratio (mrq)': 'CurrentRatio',
                      'Payout Ratio 4': 'DivPayoutRatio'
                  })

    get_last_data_item(company.fundmantal_indicators, data,
               {
                   'Return on assets': 'ROA',
                   'Return on equity': 'ROE',
                   'Total cash per share': 'Cash/Share',
                   'Book value per share': 'Book/Share',
                   'Total debt/equity': 'Debt/Equity'
               })

In [24]:
config = {}
company = Company('AAPL')
# Note:
#You might want to create an event loop and run within the loop:
loop = asyncio.get_event_loop()


In [25]:
loop.run_until_complete(get_fundamental_indicators_for_company(config, company))
print(company.fundmantal_indicators)

{'MarketCap': '2.09T', 'PS': '6.43', 'PE': '28.16', 'PEG': '1.45', 'PB': '30.23', 'ProfitMargin': '23.45%', 'OperMargin': '27.32%', 'CurrentRatio': '1.14', 'DivPayoutRatio': '18.34%', 'ROA': 16.9, 'ROE': 103.4, 'Cash/Share': 4.18, 'Book/Share': 4.15, 'Debt/Equity': 175.84}


In [26]:
data = get_statatistics(company.symbol)
data

,0,1
0,Beta (5Y Monthly),1.20
1,52-Week Change 3,62.94%
2,S&P500 52-Week Change 3,45.93%
3,52 Week High 3,145.09
4,52 Week Low 3,75.05
5,50-Day Moving Average 3,128.99
6,200-Day Moving Average 3,126.41
0,Avg Vol (3 month) 3,102.54M
1,Avg Vol (10 day) 3,90.26M
2,Shares Outstanding 5,16.69B


In [21]:
company.fundmantal_indicators = {}
# Statistics Valuation
keys = {
        'Market Cap (intraday) 5': 'MarketCap',
        'Price/Sales (ttm)': 'PS',
        'Trailing P/E': 'PE',
        'PEG Ratio (5 yr expected) 1': 'PEG',
        'Price/Book (mrq)': 'PB'
     }

In [22]:
get_data_item(company.fundmantal_indicators, data, keys)

In [23]:
get_data_item(company.fundmantal_indicators, data,
                  {
                      'Profit Margin': 'ProfitMargin',
                      'Operating Margin (ttm)': 'OperMargin',
                      'Current Ratio (mrq)': 'CurrentRatio',
                      'Payout Ratio 4': 'DivPayoutRatio'
                  })

get_last_data_item(company.fundmantal_indicators, data,
               {
                   'Return on assets': 'ROA',
                   'Return on equity': 'ROE',
                   'Total cash per share': 'Cash/Share',
                   'Book value per share': 'Book/Share',
                   'Total debt/equity': 'Debt/Equity'
               })

In [27]:
company.fundmantal_indicators

{'MarketCap': '2.09T',
 'PS': '6.43',
 'PE': '28.16',
 'PEG': '1.45',
 'PB': '30.23',
 'ProfitMargin': '23.45%',
 'OperMargin': '27.32%',
 'CurrentRatio': '1.14',
 'DivPayoutRatio': '18.34%',
 'ROA': 16.9,
 'ROE': 103.4,
 'Cash/Share': 4.18,
 'Book/Share': 4.15,
 'Debt/Equity': 175.84}